<h3>Vytvoření y</h3>

y = co má neuronka předpovídat. V našem případě začneme tím, zda byl v daném čase L-mod nebo H-mod.

Je třeba:

- načíst signály popisující začátek a konec H-módu v daném výboji: `t_H_mode_start` a `t_H_mode_end`.

- z adresáře s obrazovými daty zjistit časy snímků kamery v daném výboji

- pro každý čas zjistit, zda jde o L-mód nebo H-mód

- uložit do csv (např ve formátu slupců [název souboru se snímkem z kamery, čas ve výboji, label zda L-mode/H-mode]) - použijte pandas

Importing libraries

In [1]:
import os
os.chdir("/compass/Shared/Users/bogdanov/vyzkumny_ukol")

In [2]:
from cdb_extras import xarray_support as cdbxr   # načítání dat z databáze COMPASSu
from pyCDB import client
import numpy as np         # práce s numerickými poli
import pandas as pd
from pathlib import Path
%matplotlib inline
from tqdm.auto import tqdm


#import module with functions from the tutorial
import cdb_img_processing as img
cdb = client.CDBClient()

Importing data

In [3]:
shot = 18057
t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")

In [4]:
ris1_data = img.load_RIS_data(shot, 1) #[:34]<==test REMove after completion the rest of the task) 
ris2_data = img.load_RIS_data(shot, 2)

ris1_imgs = img.flip_image(img.demosaic(ris1_data))    #[:34]<==test REMove after completion the rest of the task) 
ris2_imgs = img.flip_image(img.demosaic(ris2_data))

In [5]:
#TODO:  To create a DataFrame with only one row, you need to specify an index, so if during the shot plasma was in H-mode more than once then next row creates different shape
t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})

In [6]:
LorH = pd.DataFrame(data={'mode':np.full(len(ris1_imgs), 'L-mode')})#, index=pd.Index(ris1_imgs.time, name='time'))#contains time and the state of the plasma (L-mode, H-mode, ELM)

In [7]:
#Nevim jak bych to udelal bez for cyklu:
for elm in t_ELM.values:
    LorH.loc[elm[0]:elm[1]] = 'ELM'

for H_mode in t_H_mode.values:
    LorH.loc[H_mode[0]:H_mode[1]] = 'H-mode'

Saving the images

In [8]:
out_path = Path('./imgs')

ris1_names = pd.DataFrame(data={'filename':img.save_ris_images_to_folder(ris1_data, path=out_path, ris=1, shot=shot)}) #I conciously decided not to cut the time in the filename at any decimal
ris2_names = pd.DataFrame(data={'filename':img.save_ris_images_to_folder(ris2_data, path=out_path, ris=2, shot=shot)})

  0%|          | 0/2100 [00:00<?, ?it/s]

  0%|          | 0/2100 [00:00<?, ?it/s]

In [9]:
lr = LorH.copy()
lr = pd.concat([LorH, ris1_names], axis=1, ignore_index=False, join='outer') #I concatenate only img names only from RIS1, since they differ only in 1 char from RIS2
lr=lr.set_index(pd.Index(ris1_imgs.time, name='time')) 

lr

,mode,filename
time,,
950.0,L-mode,imgs/RIS1_18057_t=950.0000000000001.png
950.2,L-mode,imgs/RIS1_18057_t=950.2000000000002.png
950.4,L-mode,imgs/RIS1_18057_t=950.4000000000001.png
950.6,L-mode,imgs/RIS1_18057_t=950.6000000000001.png
950.8,L-mode,imgs/RIS1_18057_t=950.8000000000001.png
...,...,...
1369.0,L-mode,imgs/RIS1_18057_t=1369.0.png
1369.2,L-mode,imgs/RIS1_18057_t=1369.2000000000003.png
1369.4,L-mode,imgs/RIS1_18057_t=1369.4.png


In [10]:
lr.to_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')